In [1]:
import os
from pymongo import MongoClient
import pandas as pd
import json
from googleapiclient.discovery import build

#### Importing wishlist videos csv

In [ ]:
wishlist_df = pd.read_csv('./csvs/modified_wish_list_xx.csv')     # last updated wishlist videos dataset

In [ ]:
wishlist_df.info()

#### Getting list of youtube_id for wishlist videos missing category_id in live dataset 

In [ ]:
wishlist_YTids_noCategoryIds_list = list(wishlist_df.loc[wishlist_df['category_id'].isna(),'youtube_id'])
wishlist_YTids_noCategoryIds_list[:5]

### YouTube API key

In [ ]:
developer_key = os.environ.get('DEVELOPER_KEY')

youtube = build('youtube', 'v3', developerKey=developer_key) # project-0127 Data Cleaning

#### Fetching category id of wishlist videos

In [ ]:
part_string = 'snippet'
videos_youtube_id = wishlist_YTids_noCategoryIds_list

In [ ]:
wishlist_categoryId_dict = {}
for id in range(0,len(videos_youtube_id)):
    print("id: ", id)
    
    try:
        request = youtube.videos().list(
            part=part_string,
            id=videos_youtube_id[id],
        )
        response = request.execute()
        
        if response['items']!=[] and response['items'][0]['snippet']['categoryId'] != '':

            wishlist_categoryId = response['items'][0]['snippet']['categoryId']
        
        else:
            wishlist_categoryId = None

    except Exception as e:
        print("An exception occured")
        print(e)
        wishlist_categoryId = None

    wishlist_categoryId_dict[videos_youtube_id[id]] = wishlist_categoryId

In [ ]:
category_id = open('./youtube_api/wishlist_category_id.txt',"a")
category_id.write(json.dumps(wishlist_categoryId_dict))
category_id.close()

### Connection to MongoDB

In [3]:
mongodb_uri = os.environ.get('MONGODB_URI')
database_name = os.environ.get('DATABASE_NAME')

client = MongoClient(mongodb_uri)
database = client[database_name]

#### Connecting to wishlist collection

In [ ]:
wishlist = database['modified_wish_list_xx']     # updated wishlist videos collection

#### Updating wishlist video category id

In [ ]:
try:
    for (key,value) in wishlist_categoryId_dict.items():
        wishlist.update_many(
        {"youtube_id" : key},
        {"$set": { "category_id" : value}}
    );
except Exception as e:
    print(e)